# Learn API

In [206]:
import requests
import numpy as np
import json

In [11]:
response = requests.get("http://api.open-notify.org/iss-now.json")

In [12]:
response.status_code

200

In [19]:
response.json()

{'message': 'success',
 'iss_position': {'longitude': '15.2324', 'latitude': '-7.7429'},
 'timestamp': 1544768878}

In [21]:
# Set up the parameters we want to pass to the API.
# This is the latitude and longitude of New York City.
parameters = {"lat": 40.71, "lon": -74}

# Make a get request with the parameters.
response = requests.get("http://api.open-notify.org/iss-pass.json", params=parameters)

# Print the content of the response (the data the server returned)
print(response.json())

# This gets the same data as the command above
response = requests.get("http://api.open-notify.org/iss-pass.json?lat=40.71&lon=-74")
print(response.json())

{'message': 'success', 'request': {'altitude': 100, 'datetime': 1544765793, 'latitude': 40.71, 'longitude': -74.0, 'passes': 5}, 'response': [{'duration': 583, 'risetime': 1544797244}, {'duration': 636, 'risetime': 1544802994}, {'duration': 566, 'risetime': 1544808855}, {'duration': 554, 'risetime': 1544814713}, {'duration': 623, 'risetime': 1544820512}]}
{'message': 'success', 'request': {'altitude': 100, 'datetime': 1544765793, 'latitude': 40.71, 'longitude': -74.0, 'passes': 5}, 'response': [{'duration': 583, 'risetime': 1544797244}, {'duration': 636, 'risetime': 1544802994}, {'duration': 566, 'risetime': 1544808855}, {'duration': 554, 'risetime': 1544814713}, {'duration': 623, 'risetime': 1544820512}]}


# Location HERE API 

In [5]:
parameters = {"app_id" : "EPimKPfenNGqsgGCr01O", "app_code" : "n6V78llT2352UUagDt295g"}

app_id = "EPimKPfenNGqsgGCr01O"
app_code  = "n6V78llT2352UUagDt295g"

In [3]:
response = requests.get("https://places.cit.api.here.com/places/v1/autosuggest?at=40.74917,-73.98529&q=chrysler",params=parameters)

In [29]:
response.status_code

200

# Finding Geocode 

In [55]:
search_results = requests.get("https://geocoder.api.here.com/6.2/geocode.json",
                             params={"app_id":app_id,
                                     "app_code":app_code,
                                     "searchtext":"Basundhara+Dhaka"})

In [56]:
latitude = search_results.json()['Response']['View'][0]['Result'][0]['Location']['NavigationPosition'][0]['Latitude']
longitude = search_results.json()['Response']['View'][0]['Result'][0]['Location']['NavigationPosition'][0]['Longitude']

# Finding Nearby Banks

In [87]:
browse_results = requests.get("https://places.cit.api.here.com/places/v1/browse",
                             params={"app_id":app_id,
                                         "app_code":app_code,
                                         "at":{"latitude":latitude,"longitude":longitude},
                                         "cat":"atm-bank-exchange"
                                         })

In [88]:
browse_results.url

'https://places.cit.api.here.com/places/v1/browse?app_id=EPimKPfenNGqsgGCr01O&app_code=n6V78llT2352UUagDt295g&at=latitude&at=longitude&cat=atm-bank-exchange'

In [89]:
browse_results.text

'{"status":400,"message":"Error in query parameter \'at\': incorrectly formatted position latitude"}'

In [91]:
name = "Mayeesha"
age = 26
s = f"Hello {name}, you are {age}"

In [99]:
browse_request = f"""https://places.cit.api.here.com/places/v1/browse?app_id={app_id}&app_code={app_code}&in={latitude},{longitude};r=2000&cat=atm-bank-exchange&pretty"""

In [101]:
browse_results2 = requests.get(browse_request)

In [107]:
json_result = browse_results2.json()

In [122]:
for result in json_result["results"]["items"]:
    print(result['title'])
    print(result['distance'])

Islami Bank-Basundhara Road
157
Dutch Bangla Bank-Basundhara Road
163
Mutual Trust Bank
222
Islami Bank-Bir Uttam Rafiqul-Islam Ave
231
Mercantile Bank-Shahid Colonel Gulzar RD
318
Bank Asia-Bashundhara Abasik Elaka
329
Islami Bank-Nadda
355
Prime Bank-Abdus Sobahna Road
369
Sonali Bank-Nadda Bazar Baridhara
377
Southeast Bank-Block B
391
Dutch Bangla Bank-Shahid Colonel Gulzar RD
401
One Bank
411
First Security Training Institute, Bosundhara R/A, Dhaka
420
Sonali Bank-Nadda
485
Dutch Bangla Bank
489
National Bank-Shahid Colonel Gulzar RD
504
Brac Bank
513
City Bank
517
Eastern Bank-Abdus Sobahan Road
539
Western Union
574


In [143]:
def get_geo_code(area,city):
    searchtext=area + "+"+city
    geo_code = requests.get("https://geocoder.api.here.com/6.2/geocode.json",
                             params={"app_id":app_id,
                                     "app_code":app_code,
                                     "searchtext":searchtext})
    if geo_code.status_code==200:
        latitude= geo_code.json()['Response']['View'][0]['Result'][0]['Location']['NavigationPosition'][0]['Latitude']
        longitude = geo_code.json()['Response']['View'][0]['Result'][0]['Location']['NavigationPosition'][0]['Longitude']
        return (latitude,longitude)
    else:
        return "Unable to get geolocation. Location Features can not be enabled."

In [161]:
def make_browse_request(latitude,longitude):
    browse_request = f"""https://places.cit.api.here.com/places/v1/browse?app_id={app_id}&app_code={app_code}&in={latitude},{longitude};r=2000&cat=atm-bank-exchange&pretty"""
    return browse_request

In [180]:
def get_browse_response(browse_request):
    response = requests.get(browse_request)
    bank_names_distance = {}
    if response.status_code==200:
        json_result = response.json()
        for result in json_result["results"]["items"]:
            bank_name = result["title"]
            distance = result["distance"]
            bank_names_distance[bank_name] = distance
    return bank_names_distance

In [181]:
geo_codes = get_geo_code(area="Basundhara",city="Dhaka")


In [182]:
browse_request  = make_browse_request(latitude=geo_codes[0],longitude=geo_codes[1])


In [183]:
geo_results = get_browse_response(browse_request)

In [195]:
geo_results

{'Islami Bank-Basundhara Road': 157,
 'Dutch Bangla Bank-Basundhara Road': 163,
 'Mutual Trust Bank': 222,
 'Islami Bank-Bir Uttam Rafiqul-Islam Ave': 231,
 'Mercantile Bank-Shahid Colonel Gulzar RD': 318,
 'Bank Asia-Bashundhara Abasik Elaka': 329,
 'Islami Bank-Nadda': 355,
 'Prime Bank-Abdus Sobahna Road': 369,
 'Sonali Bank-Nadda Bazar Baridhara': 377,
 'Southeast Bank-Block B': 391,
 'Dutch Bangla Bank-Shahid Colonel Gulzar RD': 401,
 'One Bank': 411,
 'First Security Training Institute, Bosundhara R/A, Dhaka': 420,
 'Sonali Bank-Nadda': 485,
 'Dutch Bangla Bank': 489,
 'National Bank-Shahid Colonel Gulzar RD': 504,
 'Brac Bank': 513,
 'City Bank': 517,
 'Eastern Bank-Abdus Sobahan Road': 539,
 'Western Union': 574}

In [213]:
card_preference_banks = ["Dutch Bangla","Mutual Trust"]

In [233]:
def location_scoring_banks(card_preference_banks, geo_results):
    card_bank_score = {}
    # Traverse through each of the banks that ranked high in the preferences
    for card_bank in card_preference_banks:
        current_bank_scores = []
        # if any of those banks area nearby, then get a location score for those banks
        for result in geo_results.keys():
            if card_bank in result:
                current_bank_scores.append(geo_results[result])
        # each bank may have multiple locations, so the location score for a bank is the minimum of all the location scores.
        current_bank_score_avg = np.array(current_bank_scores).min()
        # assign the location score to each of the banks from the card preference
        card_bank_score[card_bank] = current_bank_score_avg
        
    return card_bank_score

In [234]:
location_scoring_banks(card_preference_banks,geo_results)

{'Dutch Bangla': 163, 'Mutual Trust': 222}